In [10]:
import pandas as pd
import numpy as np

from pyspark.sql import SparkSession
from pyspark.sql.functions import isnan, count, collect_list

In [3]:
sp = SparkSession.builder.appName("Check taxi clean-up").getOrCreate()
sp

22/08/17 16:25:53 WARN Utils: Your hostname, J-L resolves to a loopback address: 127.0.1.1; using 192.168.34.91 instead (on interface eth0)
22/08/17 16:25:53 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/08/17 16:25:55 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


### Reading the processed data

In [37]:
df = sp.read.parquet("../data/curated/yellow/2020-7")
df.show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2020-07-01 10:25:32|2020-07-01 10:33:39|         238|          75|
|2020-07-01 10:15:11|2020-07-01 10:29:24|         230|          88|
|2020-07-01 10:30:49|2020-07-01 10:38:26|          88|         232|
|2020-07-01 10:31:26|2020-07-01 10:38:02|          37|          17|
|2020-07-01 10:09:00|2020-07-01 10:34:39|         140|          61|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



In [38]:
df.count()

568980

### Reading the raw data

In [14]:
yellow = sp.read.parquet("../data/raw/yellow/yellow-2020-07.parquet")
yellow.show(5)

+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|VendorID|tpep_pickup_datetime|tpep_dropoff_datetime|passenger_count|trip_distance|RatecodeID|store_and_fwd_flag|PULocationID|DOLocationID|payment_type|fare_amount|extra|mta_tax|tip_amount|tolls_amount|improvement_surcharge|total_amount|congestion_surcharge|airport_fee|
+--------+--------------------+---------------------+---------------+-------------+----------+------------------+------------+------------+------------+-----------+-----+-------+----------+------------+---------------------+------------+--------------------+-----------+
|       1| 2020-07-01 10:25:32|  2020-07-01 10:33:39|            1.0|          1.5|       1.0|                 N|         238|          75|           2|        8.0|  0.5|    0.5|       0.

### Loading the functions used to check

In [11]:
bike_zones = pd.read_csv("../data/raw/citi/bike stations.csv")

def drop_unknown_values(data):
    """
    Function to drop values not present in the given data dictionary
    """
    data = data.filter((data["VendorID"] == 1) | (data["VendorID"] == 2))   # Filter vendorID
    payment_list = list(range(1, 7))
    data = data.filter((data["Payment_type"].isin(payment_list)))           # Filter payment type
    ratecode_list = [1, 4, 5]                                               # Do not consider airport or group rides
    data = data.filter((data["RateCodeID"].isin(ratecode_list)))            # Filter rate code

    return data

def filter_by_limits(data):
    """
    Function to filter values set by the domain of the research question
    """
    # Consider taxis that start and end in a zone with a citibike station
    data = filter_bike_zones(data)
    # Consider taxis with 1-4 passengers
    data = data.filter((data["passenger_count"] > 0) & (data["passenger_count"] < 5))
    # Consider trip distances of 25 miles or less
    data = data.filter((data["trip_distance"] > 0) & (data["trip_distance"] < 25))
    # Consider trips which start and end in different zones
    data = data.filter((data["PULocationID"] != data["DOLocationID"]))

    return data

def filter_bike_zones(data):
    """
    Filter pick up and drop off locations w.r.t. citibike station zones
    """
    zones_list = bike_zones["Taxi area code"].to_list()
    return data.filter((data["PULocationID"].isin(zones_list)) & (data["DOLocationID"].isin(zones_list)))

def drop_columns(data):
    """
    After using extra information to filter out the data, most of the columns are unncessary
    and therefore can be dropped
    """
    columns = ["tpep_pickup_datetime", "tpep_dropoff_datetime", "trip_distance", "PULocationID", "DOLocationID"]
    return data[[columns]]

In [39]:
drop_unknown_values(yellow).count()

NameError: name 'drop_unknown_values' is not defined

In [40]:
filter_by_limits(yellow).count()

NameError: name 'filter_by_limits' is not defined

The limits of the research question narrow it down quite a lot, almost 30-40% of the data is gone and also expected since there are limited zones, trip distances, and passenger counts

In [41]:
df.dtypes

[('pickup_datetime', 'timestamp'),
 ('dropoff_datetime', 'timestamp'),
 ('PULocationID', 'bigint'),
 ('DOLocationID', 'bigint')]

## Testing Green data

In [15]:
green_raw = sp.read.parquet("../data/raw/green/green-2021-01.parquet")
green_raw.show(5)

+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|VendorID|lpep_pickup_datetime|lpep_dropoff_datetime|store_and_fwd_flag|RatecodeID|PULocationID|DOLocationID|passenger_count|trip_distance|fare_amount|extra|mta_tax|tip_amount|tolls_amount|ehail_fee|improvement_surcharge|total_amount|payment_type|trip_type|congestion_surcharge|
+--------+--------------------+---------------------+------------------+----------+------------+------------+---------------+-------------+-----------+-----+-------+----------+------------+---------+---------------------+------------+------------+---------+--------------------+
|       2| 2021-01-01 11:15:56|  2021-01-01 11:19:52|                 N|       1.0|          43|         151|            1.0|         1.01|        5.5|  0.5|    0.

In [17]:
green_cur = sp.read.parquet("../data/curated/green/2021-1/")
green_cur.show(5)

+-------------------+-------------------+-------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|trip_distance|PULocationID|DOLocationID|
+-------------------+-------------------+-------------+------------+------------+
|2021-01-01 11:15:56|2021-01-01 11:19:52|         1.01|          43|         151|
|2021-01-01 11:25:59|2021-01-01 11:34:44|         2.53|         166|         239|
|2021-01-01 11:45:57|2021-01-01 11:51:55|         1.12|          41|          42|
|2021-01-01 10:57:51|2021-01-01 11:04:56|         1.99|         168|          75|
|2021-01-01 11:35:13|2021-01-01 11:44:44|         2.34|          74|         238|
+-------------------+-------------------+-------------+------------+------------+
only showing top 5 rows



In [18]:
print(green_raw.count())
print(green_cur.count())

76518
24048


## Testing FHVHV data

In [4]:
uber_raw = sp.read.parquet("../data/raw/fhvhv/fhvhv-2021-06.parquet")
uber_raw.show(5)

+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+----+----------+-------------------+-----------------+------------------+----------------+--------------+
|hvfhs_license_num|dispatching_base_num|originating_base_num|   request_datetime|  on_scene_datetime|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|trip_miles|trip_time|base_passenger_fare|tolls| bcf|sales_tax|congestion_surcharge|airport_fee|tips|driver_pay|shared_request_flag|shared_match_flag|access_a_ride_flag|wav_request_flag|wav_match_flag|
+-----------------+--------------------+--------------------+-------------------+-------------------+-------------------+-------------------+------------+------------+----------+---------+-------------------+-----+----+---------+--------------------+-----------+--

In [19]:
uber_raw.count()

14961892

In [42]:
uber_cur = sp.read.parquet("../data/curated/fhvhv/2021-6/")
uber_cur.show(5)

+-------------------+-------------------+------------+------------+
|    pickup_datetime|   dropoff_datetime|PULocationID|DOLocationID|
+-------------------+-------------------+------------+------------+
|2021-06-01 10:45:42|2021-06-01 11:03:33|         144|         146|
|2021-06-01 10:18:15|2021-06-01 10:25:47|          49|          17|
|2021-06-01 10:33:06|2021-06-01 10:42:46|          49|         225|
|2021-06-01 10:46:27|2021-06-01 10:56:50|         225|         177|
|2021-06-01 10:48:06|2021-06-01 11:04:10|         209|          45|
+-------------------+-------------------+------------+------------+
only showing top 5 rows



In [21]:
uber_cur.count()

6245769

In [22]:
uber_cur.summary().show()

+-------+------------------+------------------+------------------+
|summary|     trip_distance|      PULocationID|      DOLocationID|
+-------+------------------+------------------+------------------+
|  count|           6245769|           6245769|           6245769|
|   mean|3.3048741889107878|143.51025550256503|144.27036366538692|
| stddev|2.4251145959589118| 77.23449225025823| 76.95879395429783|
|    min|             0.006|                 4|                 4|
|    25%|             1.576|                74|                75|
|    50%|             2.567|               146|               148|
|    75%|              4.32|               226|               226|
|    max|             24.99|               263|               263|
+-------+------------------+------------------+------------------+



## Check Citi Bike data

Check schemas and old vs new location ID differences

In [33]:
citi_jul20 = sp.read.option("header", True).csv("../data/raw/citi/202101-citibike-tripdata.csv")
citi_jul20.show(5)

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|        2513|2021-01-01 00:00:...|2021-01-01 00:42:...|            3581|Underhill Ave & L...|            40.6740123|            -73.9671457|          3581|Underhill Ave & L...|          40.6740123|          -73.9671457| 47812|  

In [34]:
citi_jul20.columns

['tripduration',
 'starttime',
 'stoptime',
 'start station id',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender']

In [24]:
citi_jan21 = sp.read.option("header", True).csv("../data/raw/citi/202101-citibike-tripdata.csv")
citi_jan21.show(5)

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|end station latitude|end station longitude|bikeid|  usertype|birth year|gender|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+--------------------+---------------------+------+----------+----------+------+
|        2513|2021-01-01 00:00:...|2021-01-01 00:42:...|            3581|Underhill Ave & L...|            40.6740123|            -73.9671457|          3581|Underhill Ave & L...|          40.6740123|          -73.9671457| 47812|  

In [29]:
citi_jan21[citi_jan21.columns[:-6]].show()

+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+
|tripduration|           starttime|            stoptime|start station id|  start station name|start station latitude|start station longitude|end station id|    end station name|
+------------+--------------------+--------------------+----------------+--------------------+----------------------+-----------------------+--------------+--------------------+
|        2513|2021-01-01 00:00:...|2021-01-01 00:42:...|            3581|Underhill Ave & L...|            40.6740123|            -73.9671457|          3581|Underhill Ave & L...|
|        2519|2021-01-01 00:00:...|2021-01-01 00:42:...|            3581|Underhill Ave & L...|            40.6740123|            -73.9671457|          3581|Underhill Ave & L...|
|        1207|2021-01-01 00:00:...|2021-01-01 00:20:...|            3144|  E 81 St & Park Ave|           40.77

In [32]:
citi_jan21.columns

['tripduration',
 'starttime',
 'stoptime',
 'start station id',
 'start station name',
 'start station latitude',
 'start station longitude',
 'end station id',
 'end station name',
 'end station latitude',
 'end station longitude',
 'bikeid',
 'usertype',
 'birth year',
 'gender']

In [35]:
citi_jan22 = sp.read.option("header", True).csv("../data/raw/citi/202201-citibike-tripdata.csv")
citi_jan22.show(5)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|55262E4365A955A2| classic_bike|2022-01-18 08:23:52|2022-01-18 08:28:18|Boerum Pl\t& Paci...|         4488.09|Clinton St & Jora...|       4605.04| 40.68848905639242|-73.99116039276123|       40.69239502|      -73.99337909|       member|
|D272F1B15D841EC0| classic_bike|2022-01-21 09:03:22|

In [36]:
citi_jan22.columns

['ride_id',
 'rideable_type',
 'started_at',
 'ended_at',
 'start_station_name',
 'start_station_id',
 'end_station_name',
 'end_station_id',
 'start_lat',
 'start_lng',
 'end_lat',
 'end_lng',
 'member_casual']

In [31]:
zones = pd.read_csv("../data/raw/citi/bike stations.csv")
zones.head()

,id,name,latitude,longtitude,Taxi area code,new_id,new_latitude,new_longtitude
0,72,W 52 St & 11 Ave,40.767272,-73.993929,50,6926.01,40.767272,-73.993928
1,79,Franklin St & W Broadway,40.719116,-74.006667,231,5430.08,40.719116,-74.006667
2,82,St James Pl & Pearl St,40.711174,-74.000165,45,5167.06,40.711174,-74.000165
3,83,Atlantic Ave & Fort Greene Pl,40.683826,-73.976323,97,4354.07,40.683826,-73.976323
4,116,W 17 St & 8 Ave,40.741776,-74.001497,68,6148.02,40.741776,-74.001497


Check citi code

In [48]:
citi_jan22_cur = sp.read.option("header", True).csv("../data/curated/citi/2022-1/")
citi_jan22.show(5)

+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|         ride_id|rideable_type|         started_at|           ended_at|  start_station_name|start_station_id|    end_station_name|end_station_id|         start_lat|         start_lng|           end_lat|           end_lng|member_casual|
+----------------+-------------+-------------------+-------------------+--------------------+----------------+--------------------+--------------+------------------+------------------+------------------+------------------+-------------+
|55262E4365A955A2| classic_bike|2022-01-18 08:23:52|2022-01-18 08:28:18|Boerum Pl\t& Paci...|         4488.09|Clinton St & Jora...|       4605.04| 40.68848905639242|-73.99116039276123|       40.69239502|      -73.99337909|       member|
|D272F1B15D841EC0| classic_bike|2022-01-21 09:03:22|

In [49]:
citi_jan22_cur = citi_jan22_cur.drop("member")

In [50]:
citi_jan22_cur.show(1)

+-------------------+-------------------+-------+-------+
|2022-01-28 17:12:46|2022-01-28 17:27:03|4519.04|3928.08|
+-------------------+-------------------+-------+-------+
|2022-01-04 22:12:48|2022-01-04 22:18:27|6140.05|5971.08|
+-------------------+-------------------+-------+-------+
only showing top 1 row



In [51]:
citi_jan22_cur.show(1)

+-------------------+-------------------+-------+-------+
|2022-01-28 17:12:46|2022-01-28 17:27:03|4519.04|3928.08|
+-------------------+-------------------+-------+-------+
|2022-01-04 22:12:48|2022-01-04 22:18:27|6140.05|5971.08|
+-------------------+-------------------+-------+-------+
only showing top 1 row

